In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.activations import *
from keras.layers import *
from keras.losses import *
from keras.optimizers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import talos as ta
%matplotlib inline

np.set_printoptions(suppress=True)

Using TensorFlow backend.


In [2]:
path = (r'C:\Users\MacalusoC\Desktop\test.csv')

data = pd.read_csv(path)

In [3]:
# scale the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

# scale the data
scaled_data = scaler.fit_transform(data)

In [4]:
# separate np array into feature and label data
scaled_features = scaled_data[:, [3, 4]]
scaled_labels = scaled_data[:, [0, 1, 2]]

# split the data into train and test groups
train_features, test_features, train_labels, test_labels = train_test_split(
    scaled_features, scaled_labels, test_size=0.2, random_state=0)

In [5]:
p = {
    'first_neuron': [8, 12, 24],
    'second_neuron': [36, 48, 62, 84],
    'third_neuron': [36, 48, 62, 84],
    'fourth_neuron': [8, 12, 24],
    'batch_size': [10, 20, 30],
    'activation': [relu, 
                   softmax,
                   selu,
                   softplus,
                   softsign,
                   tanh,
                   sigmoid,
                   hard_sigmoid,
                   exponential,
                   linear],
    'optimizer' : ['SGD',
                   'RMSprop',
                   'Adagrad',
                   'Adadelta',
                   'Adam',
                   'Adamax',
                   'Nadam',],
    'loss' : [mean_squared_error, 
              mean_absolute_error, 
              mean_absolute_percentage_error, 
              mean_squared_logarithmic_error,
              squared_hinge,hinge,
              categorical_hinge,
              logcosh,],
}

In [6]:
def ann(train_features, train_labels, test_features, test_labels, params):
    
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=2, activation=params['activation']))
    model.add(Dense(params['second_neuron'], activation=params['activation']))
    model.add(Dense(params['third_neuron'], activation=params['activation']))
    model.add(Dense(params['fourth_neuron'], activation=params['activation']))
    model.add(Dense(3, activation=params['activation']))
    model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(train_features, train_labels,
                    epochs=100,
                    batch_size=params['batch_size'],
                    # validation_split=.3,
                    verbose=0,
                    validation_data=[test_features, test_labels])
    
    # modify the output model
    return out, model

In [ ]:
# run the Talos experiment
t = ta.Scan(scaled_features, scaled_labels, 
            params=p, 
            model=ann,
            grid_downsample=.01,
            )

  0%|                                                                                         | 0/2419 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


  8%|██████▍                                                                        | 199/2419 [03:49<49:20,  1.33s/it]

In [ ]:
# accessing the results data frame
t.data.head()

# accessing epoch entropy values for each round
t.peak_epochs_df

# access the summary details
t.details

In [ ]:
# use Scan object as input
r = ta.Reporting(t)

In [ ]:
# get the number of rounds in the Scan
r.rounds()

# get the highest result ('val_acc' by default)
r.high()

# get the highest result for any metric
r.high('accuracy')

# get the round with the best result
r.rounds2high()

# get the best paramaters
r.best_params()

# get correlation for hyperparameters against a metric
r.correlate('accuracy')

In [ ]:
# a regression plot for two dimensions 
r.plot_regs()

# line plot
r.plot_line()

# up to two dimensional kernel density estimator
r.plot_kde('accuracy')

# a simple histogram
r.plot_hist(bins=50)

# heatmap correlation
r.plot_corr()

# a four dimensional bar grid
r.plot_bars('batch_size', 'accuracy', 'first_neuron', 'lr')

In [ ]:
e = ta.Evaluate(t)
e.evaluate(scaled_features, scaled_labels, folds=10, average='macro')